In [1]:
import pandas as pd

In [2]:
# Load the data
data = pd.read_csv('data/dataset.csv')

In [3]:
from langchain.document_loaders import DataFrameLoader

In [4]:
loader = DataFrameLoader(data, page_content_column="Player Bio")

In [5]:
data = loader.load()

In [6]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
from config import openai_api_key, pinecone_api_key

/Users/timothybryan/opt/anaconda3/lib/python3.9/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
OPENAI_API_KEY = openai_api_key
PINECONE_API_KEY = pinecone_api_key

In [8]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [9]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment="northamerica-northeast1-gcp"
)
index_name = "nfl-prospects-2"

In [10]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [11]:
llm = OpenAI(temperature=0.7, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [16]:
docsearch = Pinecone.from_texts([d.page_content for d in data], embeddings, index_name=index_name, metadatas=[d.metadata for d in data])

KeyboardInterrupt: 

In [13]:
query = "Tell me about Justin Fields, would you recommend drafting him?"
docs  = docsearch.similarity_search(query, include_metadata=True)

In [14]:
chain.run(input_documents=docs, question=query)

' Justin Fields is a pure dropback, strong-armed thrower who has starter talent in a vertical, downfield passing attack. He burst onto the scene as a junior-college transfer in 2012 and has clear starting-caliber traits. He has won Heisman Trophy finalist, second-team Associated Press All-American and Big Ten Offensive Player of the Year honors and has finished the year ranked sixth in the country in completion percentage. He will require patient, confidence-building play-calling and a clean pocket to function at a high level in the NFL, and how he interviews with teams could go a long way toward determining his draft status. I would recommend drafting Justin Fields as he has the potential to be a quality starter in the NFL.'

In [15]:
docs

[Document(page_content='Pure dropback, strong-armed thrower who will require patient, confidence-building play-calling and a clean pocket to function at a high level in the NFL. Has starter talent in a vertical, downfield passing attack if he can learn to take command of a huddle and continue progressing as a decision-maker. How he interviews with teams could go a long way toward determining his draft status.', metadata={}),
 Document(page_content='Burst onto the scene as a junior-college transfer in 2012 when he led the nation in completion percentage (72.5), executing a quarterback-friendly system and putting himself on the NFL radar. Concerns about his height and arm strength will limit his appeal, but at worst should be a quality backup in a precision-matchup system.', metadata={}),
 Document(page_content='Will tease evaluators with his arm and athletic talent, but has yet to prove he can throw with the precision needed to sustain a starting job in the pros. Has clear starting-cali